In [1]:
pip install transformers accelerate -U hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.8/477.8 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 45.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
from hazm import Normalizer, word_tokenize

In [12]:
normalizer = Normalizer()

def tokenize_texts(tokenizer, texts):
    normalized_texts = [normalizer.normalize(text) for text in texts]
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)
    return encodings

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
model_path = "/content/drive/MyDrive/IRproject/saved_model2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [17]:
# List of texts to classify
texts = [
    '''
    به گزارش قدس انلاین، «ترکی الفیصل» شاهزاده سعودی و رییس پیشین دستگاه اطلاعاتی این کشور گفت که سعودی ها به شدت از رفتار ایالات متحده سرخورده شده اند.

الفیصل در مصاحبه با پایگاه خبری «عرب نیوز» گفت که سعودیها احساس می کنند امریکا انان را تنها گذاشته است و معتقدند که واشنگتن باید همراه با عربستان، با انچه ان را تهدیدات منطقه خلیج فارس می نامد، مقابله کند.

وی با این ادعا که این تهدید، «نفوذ ایران در منطقه» است، مدعی شد، ایران نه تنها از «حوثیها» (انصارالله) به عنوان ابزاری «برای بر هم زدن ثبات عربستان سعودی استفاده می کند، بلکه از انها برای تاثیر گذاشتن بر امنیت و ثبات گذرگاههای دریایی بین المللی در طول دریای سرخ و خلیج فارس استفاده می کند».

بر خلاف این ادعا، کارشناسان تاکید دارند که امریکاییها با کمک سعودیها به بهانه مبارزه با دزدی دریایی و قاچاق سلاح، حضور خود را در دریای سرخ و تنگه باب المندب افزایش داده اند؛ اقدامی که امریکا از این طریق، به دنبال شکست اتش بس در یمن است.

این شاهزاده سعودی همچنین مدعی شد که حذف نام انصارالله از لیست موسوم به گروههای تروریستی، باعث شده است که حملات این جنبش به عربستان و امارات بیشتر شود. این در حالی است که مقامات یمن بارها تاکید کرده اند که این حملات، پاسخی به هشت سال جنایت ایتلاف سعودی در این کشور است.

الفیصل ادامه داد: «[جو بایدن] رییس جمهور امریکا در کمپین انتخاباتی خود گفت که عربستان سعودی را کنار خواهد گذاشت و طبیعتا با توقف عملیات مشترکی که امریکا و عربستان برای مقابله با حوثیها در یمن داشتند، به وعده خود عمل کرد».

وی با بیان اینکه بایدن همچنین گفته بود که با محمد بن سلمان ولیعهد سعودی دیدار نخواهد کرد، ادعا کرد، عربستان سعودی از مدتها پیش خواستار حل مسالمت امیز جنگ یمن است، ولی انصارالله با این مسیله موافقت نمی کند.

این ادعا در حالی مطرح شده است که با وجود گذشت چندین روز از اتش بس، ایتلاف سعودی اجازه ورود کشتیهای حامل سوخت به یمن را نمی دهد و از پرواز به فرودگاه صنعا جلوگیری می کند.

در همین ارتباط، جنبش انصارالله یمن بیست و ششم فروردین ماه جاری از افزایش تنش ها توسط امریکا در دریای سرخ، هم زمان با اجرای اتش بس سازمان ملل در این کشور انتقاد کرد.

«محمد عبدالسلام» سخنگوی رسمی این جنبش تاکید کرد: «تحرکات امریکایی در دریای سرخ و در سایه اتش بس انسانی و نظامی در یمن با ادعای واشنگتن درباره حمایت از اتش بس تناقض دارد؛ چرا که این تحرکات به دنبال تقویت وضعیت های جنگ و محاصره علیه یمن هستند».

این در حالی است که اوایل اوریل «هانس گروندبرگ» فرستاده ویژه سازمان ملل در یمن اغاز اتش بس و توقف عملیات های نظامی در این کشور را اعلام کرده بود؛ اتش بسی که شامل تمام عملیات های هوایی، زمینی و دریایی [چه در داخل یمن و چه در مرزهای خارجی ان] می شود.

بر خلاف انچه ایتلاف متجاوز سعودی_اماراتی ادعا می کند، این ایتلاف به هیچ وجه به اتش بس پایبند نیست؛ حتی شامگاه گذشته نیز عبدالسلام خبر داد که از زمان اجرایی شدن اتش بس، ایتلاف متجاوز اجازه نمی دهد هیچ هواپیمایی وارد صنعا شود و از ورود کشتیهای حامل سوخت به بندرگاه الحدیده جلوگیری می کند
    ''',
    '''
    از کارگران معاونت فرهنگی و دانشجویی قدردانی شد  به گزارش مفداابادان، در تاریخ 11اردیبهشت 1401 با حضور سعیدجلوای معاون فرهنگی و دانشجویی و مدیران این معاونت از کارگران شاغل در خدمات و تاسیسات، فضای سبز و واحدنقلیه نقلیه قدردانی شد. دوشنبه 12 اردیبهشت 1401 _ 18:12 شناسه خبر: 53848402 _ سرویس عکس
 نسخه قابل چاپ منبع: مفدا | لینک خبر از کارگران معاونت فرهنگی و دانشجویی قدردانی شد به گزارش مفداابادان، در تاریخ 11اردیبهشت 1401 با حضور سعیدجلوای معاون فرهنگی و دانشجویی و مدیران این معاونت از کارگران شاغل در خدمات و تاسیسات، فضای سبز و واحدنقلیه نقلیه قدردانی شد.
    ''',
    '''
    "normed_whole_content" : """عمل جراحی سرمربی تیم ملی در کرواسی/ اسکوچیچ زیر تیغ رفت  سرمربی تیم ملی فوتبال کشورمان که برای گذراندن تعطیلات به کرواسی بازگشته، یک جراحی موفقیت امیز را پشت سر گذاشت. مجموع نظرات: 0  صفحه اصلی بخش های دیگر اخبار چهره ها 0 نفر  دوشنبه 12 اردیبهشت 1401 _ 18:16 عمل جراحی سرمربی تیم ملی در کرواسی/ اسکوچیچ زیر تیغ رفت
سرمربی تیم ملی فوتبال کشورمان که برای گذراندن تعطیلات به کرواسی بازگشته، یک جراحی موفقیت امیز را پشت سر گذاشت.
 خبرورزشی _
دراگان اسکوچیچ پس از قرعه کشی دیدارهای تیم ملی در جام جهانی، برای دیدار با خانواده به کشورش بازگشت. وی در این اقامت، یک جراحی موفقیت امیز را نیز پشت سر گذاشت.

سرمربی تیم ملی با عارضه گوارشی مواجه بود که با این جراحی موفقیت امیز این مشکل درمان شد. دوره نقاهت اسکوچیچ چند روزه بوده و سرمربی تیم ملی هم اکنون مشکلی ندارد.
 کد خبر: 359537  0 نفر  برچسب ها دراگان اسکوچیچ برگزیده ها دلیل سکوت علی دایی در اینستاگرام چیست؟/ بی تفاوتی به پیغام ماتیوس و اخرین پست برای خانم های ایرانی! حمله شدید به سرمربی شباب الاهلی؛ تو ستاره استقلال را نابودی کردی! سند پیکان علیه پرسپولیس/ جای استوک های امید عالیشاه را ببینید! عکس| دلیل خوشحالی مدافع گلزن استقلال مشخص شد/ رمزگشایی از اعداد 3 و 4! قابل توجه فدراسیون فوتبال ایران؛ 3 بازی تدارکاتی مهم کره جنوبی مشخص شد خبرویژه دلیل سکوت علی دایی در اینستاگرام چیست؟/ بی تفاوتی به پیغام ماتیوس و اخرین پست برای خانم های ایرانی! اعتراف بزرگ رشید مظاهری؛ ببخشید کلاس سپاهان را پایین اوردم!/ حالا یک مربی هم از من خوشش نیاید! فرهاد مجیدی به رشید مظاهری طعنه زد؟ ملی پوش سپاهان استارت اشتی با استقلالی ها را قبل از بازی زد! تماس تلفنی بازیکنان سابق استقلال با گلزن این تیم پربازدیدترین برنامه پخش زنده فوتبال و مسابقات ورزشی امروز انقدر در لیگ ایران خودم را نشان می دهم تا در لالیگا سرمربی شوم/ خیلی از بازیکنان لیاقت حضور در جام جهانی را ندارند/ از 18 سالگی در تیم ملی تنها بودم! تکلیف یک لژیونر دیگر مد نظر پرسپولیس هم مشخص شد/ هرگز به ایران برنمی گردم عکس| امیر قلعه نویی بر سر مزار مطرح ترین پرسپولیسی تاریخ دربی حمله شدید به سرمربی شباب الاهلی؛ تو ستاره استقلال را نابودی کردی! گام بلند استقلال برای قهرمانی/ خوشحالی پنهان فرهاد مجیدی را هیچ کس ندید! عکس| کری خوانی عجیب بازیکن جنجالی استقلال با رنگ قرمز/ گلر ملی پوش به پرسپولیس چراغ سبز نشان داد؟ اولین اخراجی پرسپولیس مشخص شد عکس| گاف ستاره پرسپولیس بعد از بازی مقابل پیکان بازیکن پرسپولیس فقط از نظر چهره شبیه استیلی است و فوتبال بلد نیست/ سبک بازی یک بازیکن در شان این تیم نیست! نظر شما شما در حال پاسخ به نظر «» هستید. نام ایمیل نظر شما * * لطفا حاصل عبارت را در جعبه متن روبرو وارد کنید 3 + 1 = topic[id:129، title:اخبار چهره ها] _  []  topic[id:129، title:اخبار چهره ها] _  []  topic[id:129، title:اخبار چهره ها] _  []  تعمیر کرکره برقی دانلود فیلم جدید بلیط هواپیما انلاین دستگاه تصفیه هوا رزرو هتل topic[id:129، title:اخبار چهره ها] _  []  topic[id:129، title:اخبار چهره ها] _  []  پربازدیدترین برنامه پخش زنده فوتبال و مسابقات ورزشی امروز انقدر در لیگ ایران خودم را نشان می دهم تا در لالیگا سرمربی شوم/ خیلی از بازیکنان لیاقت حضور در جام جهانی را ندارند/ از 18 سالگی در تیم ملی تنها بودم! تکلیف یک لژیونر دیگر مد نظر پرسپولیس هم مشخص شد/ هرگز به ایران برنمی گردم عکس| امیر قلعه نویی بر سر مزار مطرح ترین پرسپولیسی تاریخ دربی حمله شدید به سرمربی شباب الاهلی؛ تو ستاره استقلال را نابودی کردی! گام بلند استقلال برای قهرمانی/ خوشحالی پنهان فرهاد مجیدی را هیچ کس ندید! عکس| کری خوانی عجیب بازیکن جنجالی استقلال با رنگ قرمز/ گلر ملی پوش به پرسپولیس چراغ سبز نشان داد؟ اولین اخراجی پرسپولیس مشخص شد عکس| گاف ستاره پرسپولیس بعد از بازی مقابل پیکان بازیکن پرسپولیس فقط از نظر چهره شبیه استیلی است و فوتبال بلد نیست/ سبک بازی یک بازیکن در شان این تیم نیست! topic[id:129، title:اخبار چهره ها] _  []  topic[id:129، title:اخبار چهره ها] _  []  اخرین خبر این بهترین خبر برای استقلال بود/ بازی با شهرخودرو از سپاهان هم سخت تر است/ ما حریف عشق و علاقه مردم نشدیم 55 میلیارد تومان چه زمانی به فدراسیون فوتبال می رسد؟ دلیل سکوت علی دایی در اینستاگرام چیست؟/ بی تفاوتی به پیغام ماتیوس و اخرین پست برای خانم های ایرانی! اعتراف بزرگ رشید مظاهری؛ ببخشید کلاس سپاهان را پایین اوردم!/ حالا یک مربی هم از من خوشش نیاید! فرهاد مجیدی به رشید مظاهری طعنه زد؟ ملی پوش سپاهان استارت اشتی با استقلالی ها را قبل از بازی زد!
    ''',

]

In [18]:



# Tokenize the texts
encodings = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors='pt')

# Move the input tensors to the device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_ids = encodings["input_ids"].to(device)
attention_mask = encodings["attention_mask"].to(device)

model.to(device)

# Make predictions
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

# Move the logits to the CPU and convert to NumPy array
logits = outputs.logits.detach().cpu().numpy()

# Get the predicted class labels
predicted_labels = np.argmax(logits, axis=1)

# Map the predicted labels to class names
class_names = ['Beinolmelal', 'Eghtesadi', 'Ejtemaee', 'ElmiVaDaneshghai', 'FarhangHonarVaResane', 'Siasi', 'Varzeshi']
predicted_classes = [class_names[label] for label in predicted_labels]

# Print the results
for text, predicted_class in zip(texts, predicted_classes):
    print(f"Predicted Class: {predicted_class}")
    print()


Predicted Class: Beinolmelal

Predicted Class: ElmiVaDaneshghai

Predicted Class: Varzeshi

